In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

def func_complex_fields(df):
    complex_fields = dict([(field.name, field.dataType)
                            for field in df.schema.fields
                            if(type(field.dataType) == ArrayType or type(field.dataType) == StructType)
                            ])
    return complex_fields

def normalization(df):
    complex_fields = func_complex_fields(df)
    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        print(f'Processing: {col_name} Type: {str(type(complex_fields[col_name]))}')
        # Se for StructType então vamos converter todos os sub elementos em colunas        
        if(type(complex_fields[col_name]) == StructType):
            expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [n.name for n in complex_fields[col_name]]]
            df = df.select("*", *expanded).drop(col_name)

        elif(type(complex_fields[col_name]) == ArrayType):
            df = df.withColumn(col_name, explode_outer(col_name))

        complex_fields = func_complex_fields(df)
        
    return df

In [0]:
dfTrusted = spark.sql("""   
    select 
        md5(concat(coalesce(id, '-')
            ,coalesce(url, '-')
            ,coalesce(categories_id, '-')
            ,coalesce(categories_name, '-') 
            ))                                          as id_chave
    ,CAST(height                     as int)            as vl_height
    ,CAST(id                         as string)         as ch_id
    ,CAST(url                        as string)         as ch_url
    ,CAST(width                      as int)            as vl_width
    ,CAST(breeds_adaptability        as int)            as vl_breeds_adaptability
    ,CAST(breeds_affection_level     as int)            as vl_breeds_affection_level
    ,CAST(breeds_alt_names           as string)         as ch_breeds_alt_names
    ,CAST(breeds_bidability          as int)            as vl_breeds_bidability
    ,CAST(breeds_cat_friendly        as int)            as vl_breeds_cat_friendly
    ,CAST(breeds_cfa_url             as string)         as ch_breeds_cfa_url
    ,CAST(breeds_child_friendly      as int)            as vl_breeds_child_friendly
    ,CAST(breeds_country_code        as string)         as ch_breeds_country_code
    ,CAST(breeds_country_codes       as string)         as ch_breeds_country_codes
    ,CAST(breeds_description         as string)         as ch_breeds_description
    ,CAST(breeds_dog_friendly        as int)            as vl_breeds_dog_friendly
    ,CAST(breeds_energy_level        as int)            as vl_breeds_energy_level
    ,CAST(breeds_experimental        as int)            as vl_breeds_experimental
    ,CAST(breeds_grooming            as int)            as vl_breeds_grooming
    ,CAST(breeds_hairless            as int)            as vl_breeds_hairless
    ,CAST(breeds_health_issues       as int)            as vl_breeds_health_issues
    ,CAST(breeds_hypoallergenic      as int)            as vl_breeds_hypoallergenic
    ,CAST(breeds_id                  as string)         as ch_breeds_id
    ,CAST(breeds_indoor              as int)            as vl_breeds_indoor
    ,CAST(breeds_intelligence        as int)            as vl_breeds_intelligence
    ,CAST(breeds_lap                 as int)            as vl_breeds_lap
    ,CAST(breeds_life_span           as string)         as ch_breeds_life_span
    ,CAST(breeds_name                as string)         as ch_breeds_name
    ,CAST(breeds_natural             as int)            as vl_breeds_natural
    ,CAST(breeds_origin              as string)         as ch_breeds_origin
    ,CAST(breeds_rare                as int)            as vl_breeds_rare
    ,CAST(breeds_reference_image_id  as string)         as ch_breeds_reference_image_id
    ,CAST(breeds_rex                 as int)            as vl_breeds_rex
    ,CAST(breeds_shedding_level      as int)            as vl_breeds_shedding_level
    ,CAST(breeds_short_legs          as int)            as vl_breeds_short_legs
    ,CAST(breeds_social_needs        as int)            as vl_breeds_social_needs
    ,CAST(breeds_stranger_friendly   as int)            as vl_breeds_stranger_friendly
    ,CAST(breeds_suppressed_tail     as int)            as vl_breeds_suppressed_tail
    ,CAST(breeds_temperament         as string)         as ch_breeds_temperament
    ,CAST(breeds_vcahospitals_url    as string)         as ch_breeds_vcahospitals_url
    ,CAST(breeds_vetstreet_url       as string)         as ch_breeds_vetstreet_url
    ,CAST(breeds_vocalisation        as int)            as vl_breeds_vocalisation
    ,CAST(breeds_wikipedia_url       as string)         as ch_breeds_wikipedia_url
    ,CAST(categories_id              as string)         as ch_categories_id
    ,CAST(categories_name            as string)         as ch_categories_name
    ,CAST(breeds_weight_imperial     as string)         as ch_breeds_weight_imperial
    ,CAST(breeds_weight_metric       as string)         as ch_breeds_weight_metric
    ,CAST(dt_referencia_criacao      as timestamp)      as dt_referencia_criacao
    ,CAST(dt_referencia_atualizacao  as timestamp)      as dt_referencia_atualizacao
from vw_trusted
""")

In [0]:
(dfTrusted
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", True)
    .save('/dbfs/tmp/dbtrusted/tbCat')
)
spark.sql("create table if not exists dbtrusted.tbcat using delta location '/dbfs/tmp/dbtrusted/tbCat'")

Out[19]: DataFrame[]

In [0]:
%sql
select * from dbtrusted.tbcat

id_chave,vl_height,ch_id,ch_url,vl_width,vl_breeds_adaptability,vl_breeds_affection_level,ch_breeds_alt_names,vl_breeds_bidability,vl_breeds_cat_friendly,ch_breeds_cfa_url,vl_breeds_child_friendly,ch_breeds_country_code,ch_breeds_country_codes,ch_breeds_description,vl_breeds_dog_friendly,vl_breeds_energy_level,vl_breeds_experimental,vl_breeds_grooming,vl_breeds_hairless,vl_breeds_health_issues,vl_breeds_hypoallergenic,ch_breeds_id,vl_breeds_indoor,vl_breeds_intelligence,vl_breeds_lap,ch_breeds_life_span,ch_breeds_name,vl_breeds_natural,ch_breeds_origin,vl_breeds_rare,ch_breeds_reference_image_id,vl_breeds_rex,vl_breeds_shedding_level,vl_breeds_short_legs,vl_breeds_social_needs,vl_breeds_stranger_friendly,vl_breeds_suppressed_tail,ch_breeds_temperament,ch_breeds_vcahospitals_url,ch_breeds_vetstreet_url,vl_breeds_vocalisation,ch_breeds_wikipedia_url,ch_categories_id,ch_categories_name,ch_breeds_weight_imperial,ch_breeds_weight_metric,dt_referencia_criacao,dt_referencia_atualizacao
a283653178e29e1169b305b2be19e809,960,9l6,https://cdn2.thecatapi.com/images/9l6.jpg,720,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2024-10-01T16:41:27.633+0000,2024-10-01T16:43:04.279+0000
d17eed253949bbc5e76c1ccd59970acd,466,q5,https://cdn2.thecatapi.com/images/q5.jpg,700,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6,caturday,null,null,2024-10-01T16:41:27.633+0000,2024-10-01T16:43:04.279+0000
7f67ec0691dbf20e6821bbbcba4ac9ac,304,4he,https://cdn2.thecatapi.com/images/4he.gif,400,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2024-10-01T16:41:27.633+0000,2024-10-01T16:43:04.279+0000
855a3a527fd67e575e583459461d6a71,333,MTc5OTExMQ,https://cdn2.thecatapi.com/images/MTc5OTExMQ.jpg,500,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,15,clothes,null,null,2024-10-01T16:41:27.633+0000,2024-10-01T16:43:04.279+0000
a09f84d5a7f90cf3340083c054769960,313,dpa,https://cdn2.thecatapi.com/images/dpa.jpg,500,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2024-10-01T16:41:27.633+0000,2024-10-01T16:43:04.279+0000
2985977449e1fbc902f41bc51f11b855,500,2re,https://cdn2.thecatapi.com/images/2re.jpg,375,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2024-10-01T16:41:27.633+0000,2024-10-01T16:43:04.279+0000
3c8fa30788a4fa60bc3064d08c0a6121,3264,lOl0J96On,https://cdn2.thecatapi.com/images/lOl0J96On.jpg,2448,3,4,,4,3,http://cfa.org/Breeds/BreedsCJ/ColorpointShorthair.aspx,4,US,US,"Colorpoint Shorthairs are an affectionate breed, devoted and loyal to their people. Sensitive to their owner’s moods, Colorpoints are more than happy to sit at your side or on your lap and purr words of encouragement on a bad day. They will constantly seek out your lap whenever it is open and in the moments when your lap is preoccupied they will stretch out in sunny spots on the ground.",4,4,0,2,0,2,0,csho,0,5,1,12 - 16,Colorpoint Shorthair,0,United States,0,oSpqGyUDS,0,3,0,4,2,0,"Affectionate, Intelligent, Playful, Social",https://vcahospitals.com/know-your-pet/cat-breeds/colorpoint-shorthair,null,5,https://en.wikipedia.org/wiki/Colorpoint_Shorthair,null,null,4 - 10,2 - 5,2024-